# 1. Import all the packages we need

In [1]:
import pickle
import numpy as np
import os
import re
import json
import pandas as pd
from Ant import Ant

# 2. Run the following code to make sure all the functions can work

In [2]:
def compute_distance2(start_x, start_y, tar_x, tar_y):
    """
    compute the distance between two points
    """
    return 2 * (abs(start_x - tar_x) + abs(start_y - tar_y))

def compute_distance3(start_x, start_y, mid_x, mid_y, tar_x, tar_y):
    """
    compute the distance between three points
    """
    dis1 = 2 * (abs(start_x - mid_x) + abs(start_y - mid_y))
    dis2 = 2 * (abs(mid_x - tar_x) + abs(mid_y - tar_y))
    return (dis1, dis2, dis1 + dis2)

def get_path(points, stay_time, wind_matrix, rain_matrix, wind_threshold, rain_threshold):
    """
    get the path at the guide of points and stay time
    
    points -- guide points to help the Ant find the path
    stay_time -- stay time at each point, mostly used to change the take-off time of the plane
    wind_matrix -- all the wind value at each point. Shape: (548, 421, 18)
    rain_matrix -- all the rainfall value at each point. Shape: (548, 421, 18)
    wind_threshold -- wind threshold at each segment path
    rain_threshold -- rain threshold at each segment path
    """
    if (len(points) != len(stay_time)):
        raise Exception("the length of points and stay_time must be equal")
    
    if (len(points) < 2):
        raise Exception("the length of points or stay_time should be bigger than 1")
    
    dis = 0
    path = [points[0]]
    for p_index in range(len(points) - 1):
        # stay at one poinst
        dis += stay_time[p_index]
        path = path + [points[p_index]] * (stay_time[p_index] // 2)
        print ('point {}: stay for {} min'.format(points[p_index], stay_time[p_index]))
        
        # shortest path
        ant = Ant(points[p_index][0], points[p_index][1], points[p_index + 1][0], points[p_index + 1][1], 
                  wind_matrix=wind_matrix, rain_matrix=rain_matrix, distance=dis, wind_threshold=wind_threshold[p_index], 
                  rain_threshold=rain_threshold[p_index])
        tmp_path = ant.has_shortest_path()
        cost_time = 2 * (len(tmp_path) - 1)
        if tmp_path:
            path = path + tmp_path[1:]
            dis += cost_time
            print ('path {}: cost {} min'.format(p_index + 1, cost_time))
        else:
            print ('failed in path between {}, {}'.format(points[p_index], points[p_index + 1]))
            print ('please retune the parameters of this path and rerun the code')
            return False
        
    print ('reach {}, cost {}min'.format(points[-1], dis))
    return path

# 3. set some paramters to guide the Ant find the path we want

### Actually, we will start a new ipython notebook for every new path in this competition, which is easier to analyze the result.

### In this notebook, we combined all the parameters in one notebook, to make sure the reader can get the all the paths as fast as possible.

### The parameter tuning process is the most tiring part of our algorithm, you can skip this step and just run the code. We have two ways to find the guide points. One is automatic and the other is manual. 

### See the PointsFinde.py file to know the automatic way

### See the django project to know the manual way

### Both of them is fast and convenient once you get the idea of what's going on behind these codes.

### the parameters are stored in json format, so you don't need to find the guide points yourself, each path in each day has different parameters, for example:

'day10path1': {

     'guide_points': [[137, 244], [139, 234]],
      'rain_matrix': 'rain_matrix_xgb.pickle',
      'rain_threshold': [0.2, 2.0, 2.0],
      'stay_time': [10, 0, 0, 0],
      'wind_matrix': 'wind_matrix_lgb.pickle',
      'wind_threshold': [9.7, 9.0, 8.0]
}

The guide points in this path is (137, 244) and (139, 234).

the used rain map is rain_matrix_xgb.pickle

the rain threshold between start point and (137, 244) is 0.2, the rain threshold between (137, 244) and (139, 234) is 2.0, 
the rain threshold between (139, 234) and target point is 2.0,  

start at 10min, and does not stay at any other point

the used wind map is wind_matrix_lgb.pickle

the wind threshold between start point and (137, 244) is 9.7, the wind threshold between (137, 244) and (139, 234) is 9.0, 
the wind threshold between (139, 234) and target point is 8.0,  

In [3]:
with open('parameters.json', 'r') as f:
    params = json.load(f)
params

{'day10path1': {'guide_points': [],
  'rain_matrix': 'rain_matrix_lgb.pickle',
  'rain_threshold': [0.5],
  'stay_time': [0, 0],
  'wind_matrix': 'wind_matrix_lgb.pickle',
  'wind_threshold': [10]},
 'day10path10': {'guide_points': [],
  'rain_matrix': 'rain_matrix_lgb.pickle',
  'rain_threshold': [1.0],
  'stay_time': [400, 0],
  'wind_matrix': 'wind_matrix_lgb.pickle',
  'wind_threshold': [9.0]},
 'day10path2': {'guide_points': [],
  'rain_matrix': 'rain_matrix_lgb.pickle',
  'rain_threshold': [0.5],
  'stay_time': [90, 0],
  'wind_matrix': 'wind_matrix_lgb.pickle',
  'wind_threshold': [10]},
 'day10path3': {'guide_points': [[137, 244], [139, 234]],
  'rain_matrix': 'rain_matrix_lgb_time.pickle',
  'rain_threshold': [0.2, 2.0, 2.0],
  'stay_time': [10, 0, 0, 0],
  'wind_matrix': 'wind_matrix_lgb.pickle',
  'wind_threshold': [9.7, 9.0, 8.0]},
 'day10path4': {'guide_points': [[230, 242], [233, 239]],
  'rain_matrix': 'rain_matrix_lgb_time.pickle',
  'rain_threshold': [2.5, 4, 13],
  's

# 4. Run the code to find the path we want

### Because the map generated in diffenrent devices will be slightly different, so the path generated will be different. But we are sure most of them are still safe.

### we set the wind threshold and rain threshold really really low to ensure its a safe path, so you may fail in some paths in different maps. It's normal and okay just release the threshold a little bit, and run again.

### you can change the threshold in the json file directly or modify and run the annoted code

### Or you can run the codes on the map we generated with the same codes we offered
### We stored the map at https://pan.baidu.com/s/1i7cS041

### 我们将风速和降雨量的阈值设置的非常小，以确保路径尽可能的安全。但这是在我们的地图上所设置的参数，在不同机子上跑出来的地图是不一样的，也就是用这组参数在其它机子上跑出来的地图上无法找到相同的路径（耗时相同且安全的路径有很多）。

### 比如：在我们机子上跑出来的地图在某条路径上的最大风速为9.8，我们就将阈值掐到了10.0。此时，若在另一幅地图上，该路径的最大风速变成了10.1，这样就走不通了。所以，这个时候就要适当地提高一下风速的阈值，比如改到10.5。

### 这些参数要根据不同的地图作调整，我们无法保证这组参数适用于其它机子跑出来的地图上。需要逐条修改。

### 另一个方便的方法就是直接用我们的机子上跑出来的地图来跑下面的程序。地图是用相同的方法跑出来的。我们已经将地图上传到了百度网盘上（https://pan.baidu.com/s/1i7cS041）

In [4]:
# change the day and path to the one you want, make sure the length of the threshold is the same as before
# params['day6path1']['wind_threshold'] = [15.0, 15.0]
# params['day6path1']['rain_threshold'] = [4.0, 4.0]

In [5]:
# get city data
city_data = pd.read_csv('../dataset/CityData.csv')

# get start position
start_x, start_y = city_data['xid'][0], city_data['yid'][0]

count = 0
for day in range(6, 11, 1):
    for tar_id in range(1, 11, 1):
        # get target position
        tar_x, tar_y = city_data['xid'][tar_id], city_data['yid'][tar_id]
        
        s = 'day' + str(day) + 'path' + str(tar_id)
        dir_path = '../dataset/day' + str(day)
        
        # get wind matrix and rain matrix
        wind_file = os.path.join(dir_path, params[s]["wind_matrix"])
        rain_file = os.path.join(dir_path, params[s]["rain_matrix"])
        with open(wind_file, 'rb') as f:
            wind_matrix = pickle.load(f)
        with open(rain_file, 'rb') as f:
            rain_matrix = pickle.load(f)
        
        print ('start {}'.format(s))
        points = [(start_x,start_y)] + params[s]["guide_points"] + [(tar_x, tar_y)]
        stay_time = params[s]["stay_time"]
        wind_threshold = params[s]["wind_threshold"]
        rain_threshold = params[s]["rain_threshold"]
        path = get_path(points, stay_time, wind_matrix, rain_matrix, wind_threshold, rain_threshold)
        if path:
            count += 1
            with open('../dataset/result/' + s + '.pickle', 'wb') as f:
                pickle.dump(path, f, protocol=2)
        print ('\n')
print ('There are {}/50 paths generated'.format(count))

start day6path1
point (142, 328): stay for 496 min
path 1: cost 316 min
point [109, 203]: stay for 0 min
path 2: cost 50 min
reach (84, 203), cost 862min


start day6path2
point (142, 328): stay for 394 min
path 1: cost 200 min
reach (199, 371), cost 594min


start day6path3
point (142, 328): stay for 870 min
path 1: cost 192 min
reach (140, 234), cost 1062min


start day6path4
point (142, 328): stay for 688 min
path 1: cost 362 min
reach (236, 241), cost 1050min


start day6path5
point (142, 328): stay for 606 min
path 1: cost 440 min
reach (315, 281), cost 1046min


start day6path6
point (142, 328): stay for 0 min
path 1: cost 236 min
point [210, 378]: stay for 0 min
path 2: cost 468 min
point [325, 259]: stay for 0 min
path 3: cost 124 min
point [334, 206]: stay for 0 min
path 4: cost 50 min
reach (358, 207), cost 878min


start day6path7
point (142, 328): stay for 138 min
path 1: cost 76 min
point [152, 356]: stay for 6 min
path 2: cost 166 min
point [214, 335]: stay for 0 min
path